# Energy System Capital Cost Analysis 

## Objective 

How much an energy system cost is a very important standard to judge if this energy system is good or not? In this example, I will discover the power of mathematical optimization in addressing a common energy industry problem: energy system capital cost analysis. 

The current ERCOT solar grid will be evaluated based on the historical Houston Hub settlement point price along with the solar output in 2019.

---
## Problem Description

In this problem, a renewable energy developer want to make an investment to a new solar farm in Houston. However, they want to know what kind of configurations is most economical and beneficial for them. Therefore, an analysis on how much they would earn in 2019 by comparing three different configurations is implemented. Here, those three configurations are respectively (1) standalone storage system; (2) solar+storage system (3) standalone solar. 

The data set of 8760 hourly solar irradiance and Houston hub electricity price is as follows:

| weather year hour beginning | Solar Shape (Normalized) | Houston hub Settlement Point Energy Price (\$/MWh) |
| --- | --- | ---| 
| 1/1/2019 0:00| 0 | 14.1900	 | 
| 1/1/2019 1:00 | 0 | 15.1825	 |
| 1/1/2019 2:00 | 0 | 15.5300 | 


(1) Proposed Installation (Standalone Solar) Setting:

| Parameter | Value| Unit |
| --- | --- | --- | 
| Solar Size | 300 |  MW | 
| Inverter Capacity | 250 | MW |
| Solar System Cost | \$1.35 | \$/Watt-DC |


(2) Proposed Installation (Standalone Storage) Setting:

| Parameter | Value| Unit |
| --- | --- | --- | 
| Storage Capacity | 50 |  MW | 
| Storage Energy | 200 | MWh |
| Round Trip Efficiency | 100\% |   |
| Storage System Cost | \$375 | \$/KWh  |

(3) Proposed Installation (Combined Solar + Storage System) Setting:

| Parameter | Value| Unit |
| --- | --- | --- | 
| Solar Size | 300 |  MW | 
| Inverter Capacity | 250 | MW |
| Storage Capacity | 50 |  MW |
| Storage Energy | 200 | MWh  |
| Round Trip Efficiency | 100\% |  |
| Solar System Cost | \$1.25 | \$/Watt-DC  |
| Storage System Cost | \$350 | \$/KWh  |

---

## Model Formulation

### Parameters
$\text{RTM} \in \mathbb{R}^+$: Real Time Market Price  \$/KWh.

normSolar: Normalized Solar Shape.

timeStamp: hours begining in year 2019 using Pacific Standard Time.

Life cycle of Solar Pannel: assume 30 years in service

Life cycle of Battery Storage: assume 10 years in service

$\eta$: Round Trip Efficiency

$SOC_t$: State of Charge at hour t

$AC_t$: AC output of solar energy at hour t

$DC_t$: DC output of solar energy at hour t

$P_t$: Real time price at hour t

### Decision Variables

$SOC_t$: State of Charge at hour t

$E_t^{in}$: Energy delivered to the storage at hour t

$E_t^{out}$: Energy discharged to the grid at hour t

$E_t^{Gin}$ : Energy charged to the storage from grid at hour t

$E_t^{BoutG}$ : Energy discharged to the grid from storage at hour t

$E_t^{Sin}$ : Energy delivered to the storage from solar at hour t

$E_t^{SoutG}$   : Energy delivered to the grid from solar at hour t


### Objective Function

- **𝑅𝑒𝑣𝑒𝑛𝑢𝑒**: Maximize the revenue of solar system (in USD).

For a standalone solar system (Assume 30 years solar plant life):

\begin{equation}
\text{Maximize}  AC_t \times P_t
\end{equation}

For a standalone storage system (Assume 10 years storage life):

\begin{equation}
\text{Maximize}  E_t^{out} \times P_t - E_t^{in} \times P_t
\end{equation}

For a solar and storage system (Assume 10 years storage life, 30 years solar plant life):

\begin{equation}
\text{Maximize} (E_t^{BoutG}+E_t^{SoutG})∗P_t− E_t^{Gin}∗P_t
\end{equation}

### Constraints

For a standalone solar system:

$DC_t$ = 300*normSolar

$AC_t$ = min(250, $DC_t$)

For a standalone storage system:
(1) at any time t, the SOC will follow
\begin{equation}
SOC_t=SOC_{t-1}+\sqrt{\eta}\times E_{t-1}^{in}-E_{t-1}^{out}/\sqrt{\eta}
\end{equation}

(2) $\forall t$, $0<=SOC_t<=200$.

(3) $\forall t$, $0<=\sqrt{\eta}\times E_{t}^{in}<=50$.

(4) $\forall t$, $0<=E_{t}^{out}/\sqrt{\eta}<=50$.

(5) $\forall t$, $0<=E_{t}^{out}/\sqrt{\eta}<=SOC_t$.

For a system combined solar and battery storage:

(1) at any time t, the SOC will follow
\begin{equation}
SOC_t=SOC_{t-1}+ (E_{t-1}^{Gin}+E_{t-1}^{Sin})-E_{t-1}^{BoutG}
\end{equation}

(2) $\forall t$, $0<=SOC_t<=200$.

(3) $\forall t$, $0<=(E_{t}^{Gin}+E_{t}^{Sin}) <=50$.

(4)  $\forall t$, $0<=E_{t}^{BoutG}<=50$.

(5)  $\forall t$, $0<=E_{t}^{BoutG}<=SOC_t$.

---
## Python Implementation

We import the Gurobi Python Module and other Python libraries.

In [1]:
import pandas as pd
import numpy as np
import dataframe_image as dfi
import datetime
from pyomo.environ import *

## Input Data
We define all the input data of the model.

In [2]:
file_loc = "./case/case/price/pricee.xlsx"

dti = pd.date_range("2019-01-01", periods=8760, freq="H")

sheets_dict = pd.read_excel(file_loc, sheet_name=None)

dflist=[]
for i in sheets_dict.values():
    dfnewprice = i.loc[i['Settlement Point Name'] == 'HB_HOUSTON']
    df3 = dfnewprice['Settlement Point Price']
    df3.index = range(len(df3.index))
    df6 = df3.groupby(df3.index // 4).mean()
    dflist.append(df6)

dfhourprice = pd.concat(dflist)
dfhourprice.index = dti  
dfhourprice = dfhourprice.to_frame()

# solar shape
file_loc = "./case/case/FuelMixReport_PreviousYears/IntGenbyFuel2019.xlsx"
sheetname= ['Jan', 'Feb', 'Mar', 'Apr', 'May','Jun', 'Jul', 'Aug','Sep', 'Oct', 'Nov', 'Dec']

dfqqlist=[]

for i in sheetname:
    df_price_map = pd.read_excel(file_loc, sheet_name=i)
    dfnewprice = df_price_map.loc[df_price_map['Fuel'] == 'Solar']
    dfnewprice1 = dfnewprice.iloc[: , 4:]
    dfnewprice1 = dfnewprice1.groupby(np.arange(len(dfnewprice1.columns))//4, axis=1).mean()
    dfnewprice1.index = range(len(dfnewprice1.index))
    dfnewprice1_transposed = dfnewprice1.T # or df1.transpose(
    dfnewprice1_transposed1 = pd.Series(dfnewprice1_transposed.values.ravel('F'))
    dfqqlist.append(dfnewprice1_transposed1)

dfnormsolar = pd.concat(dfqqlist)

normalized_dfnormsolar=dfnormsolar/dfnormsolar.max()
normalized_dfnormsolar.index = dti

df = dfhourprice.merge(normalized_dfnormsolar.rename('SolarShape'), left_index=True, right_index=True)

## Compute the monthly averaged historical  Settlement Point Price for the year of 2019

In [3]:
monthly_averages = df.groupby(pd.PeriodIndex(df.index, freq="M"))['Settlement Point Price'].mean()
hourly_averages = df.groupby([df.index.month, df.index.hour])['Settlement Point Price'].mean()
dfavg1 = pd.DataFrame()

for i in range (1,13):
    for j in range (24):
        dftemp=df.loc[(df.index.month==i) & (df.index.hour==j), 'Settlement Point Price'] 
        dfavg1.loc[i-1,j+1] = round(dftemp.mean())
dfavg1.index = ['01', '02', '03', '04','05','06','07','08','09','10','11','12']
dfavg1.loc['Avg'] = round(dfavg1.mean(axis=0))

dfavg1['Avg'] = round(dfavg1.mean(axis=1))
dfavg1 = dfavg1.astype(int)
df_styled = dfavg1.style.background_gradient(axis=None, vmin=10, vmax=50, cmap='coolwarm')
dfi.export(df_styled,"Monthly_Average_Price.png")
dfavg1.to_excel("avgRTM.xlsx")
df_styled

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,Avg
01,18,17,17,18,19,22,37,29,30,30,26,26,22,21,21,20,21,32,31,26,26,24,22,19,24
02,18,16,16,17,17,19,34,26,27,35,33,23,22,22,21,20,20,22,31,24,25,24,20,19,23
03,16,15,15,16,19,33,59,29,29,30,28,28,26,28,25,24,25,26,53,30,31,25,20,18,27
04,14,14,13,14,17,43,25,20,25,23,24,23,27,39,37,42,34,32,31,43,24,21,18,17,26
05,16,15,15,16,17,22,20,22,21,25,27,31,42,51,48,39,29,26,27,25,24,21,18,18,26
06,16,15,14,15,16,16,17,18,20,22,25,30,36,41,69,90,48,26,27,23,21,19,18,17,27
07,16,15,14,14,15,16,17,18,19,22,25,45,51,63,88,91,47,27,25,25,22,20,18,18,30
08,15,14,13,13,14,15,16,18,19,22,30,38,164,599,928,828,258,44,26,25,21,19,17,16,132
09,16,15,15,15,16,18,18,20,24,26,28,43,77,121,199,299,99,32,34,27,22,21,18,17,51
10,15,14,13,13,15,33,26,19,20,21,22,24,27,54,124,96,42,25,25,21,22,18,19,16,30


In [4]:
sp1={'Solar Size': 300.0, 'Inverter Capacity': 250.0, 'Solar System Cost': 1.35}

sp2={'Storage Capacity': 50,
 'Storage Energy': 200,
 'Round Trip Efficiency': 1,
 'Storage System Cost': 375}

sp3={'Solar Size': 300.0,
 'Inverter Capacity': 250.0,
 'Storage Capacity': 50.0,
 'Storage Energy': 200.0,
 'Round Trip Efficiency': 1.0,
 'Solar System Cost': 1.25,
 'Storage System Cost': 350.0}

### case 1: Calculate the revenue of Standalone Solar configuration

In [5]:
df['SolarDC'] = df['SolarShape'].values * sp1['Solar Size']
df['SolarAC'] = df['SolarDC'].apply(lambda x: x if x <= sp1['Inverter Capacity'] else sp1['Inverter Capacity'])
df['hour'] = range(0,8760)
df.index = df.hour
results_df = df 
results_df['revenue'] = df.SolarAC * df['Settlement Point Price'] 
serviceLifeSolar = 30

total_revenue = results_df.revenue.sum()
capex = sp1['Solar Size']*sp1['Solar System Cost']*1000000/serviceLifeSolar

print('Annual revenue was ${:,.2f}'.format(total_revenue))
print('Annual cost ${:,.2f}'.format(capex))

Annual revenue was $45,577,726.99
Annual cost $13,500,000.00


### Case 2: Calculate the revenue of Standalone Storage configuration

In [6]:
def optimizeStorage(df, first_model_hour=0, last_model_hour=8759, setup=sp2):
    df = df.loc[first_model_hour:last_model_hour, :]

    model = ConcreteModel()

    # Define model parameters
    model.T = Set(doc='hour of year', initialize=df.hour.tolist(), ordered=True)
    model.Rmax = Param(initialize=setup['Storage Capacity'],
                       doc='Max rate of power flow (MW) in or out')
    model.Smax = Param(initialize=setup['Storage Energy'], doc='Max storage (MWh)')
    model.Dmax = Param(initialize=600, doc='Max discharge in 24 hour period')
    model.P = Param(initialize=df['Settlement Point Price'].tolist(), doc='RTMP for each hour')
    eta = 1 # Round trip storage efficiency

    # Charge, discharge, and state of charge
    model.Ein = Var(model.T, domain=NonNegativeReals)
    model.Eout = Var(model.T, domain=NonNegativeReals)
    model.S = Var(model.T, bounds=(0, model.Smax))


    #Set all constraints
    def storage_state(model, t):
        'Storage changes with flows in/out and efficiency losses'
        # Set first hour state of charge to half of max 
        # this can be change in to 0
        if t == model.T.first():
            # return model.S[t] == model.Smax / 2
            return model.S[t] == 0

        else:
            return (model.S[t] == (model.S[t-1] 
                                + (model.Ein[t-1] * np.sqrt(eta)) 
                                - (model.Eout[t-1] / np.sqrt(eta))))

    model.charge_state = Constraint(model.T, rule=storage_state)

    def discharge_constraint(model, t):
        "Maximum dischage within a single hour"
#         return model.Eout[t] <= model.Rmax
        return model.Eout[t]/ np.sqrt(eta) <= model.Rmax

    model.discharge = Constraint(model.T, rule=discharge_constraint)

    def charge_constraint(model, t):
        "Maximum charge within a single hour"
        return model.Ein[t]*np.sqrt(eta) <= model.Rmax

    model.charge = Constraint(model.T, rule=charge_constraint)

    def positive_charge(model, t):
        'Limit discharge to the amount of charge in battery, including losses'
        return model.Eout[t]/ np.sqrt(eta) <= model.S[t] 
    model.positive_charge = Constraint(model.T, rule=positive_charge)

    # Define the battery income, expenses, and profit
    income = sum(df.loc[t, 'Settlement Point Price' ] * model.Eout[t] for t in model.T)
    expenses = sum(df.loc[t, 'Settlement Point Price'] * model.Ein[t] for t in model.T)
    profit = income - expenses
    model.objective = Objective(expr=profit, sense=maximize)

    # Solve the model
    solver = SolverFactory('glpk')
    solver.solve(model)

    hours = range(model.T[first_model_hour + 1], model.T[last_model_hour + 1] + 1)
    Ein = [value(model.Ein[i]) for i in hours]
    Eout = [value(model.Eout[i]) for i in hours]
    Price = [model.P.extract_values()[None][i] for i in hours]
    charge_state = [value(model.S[i]) for i in hours]

    df_dict = dict(
        hour=hours,
        Ein=Ein,
        Eout=Eout,
        price=Price,
        charge_state=charge_state
    )

    results_df = pd.DataFrame(df_dict)

    results_df['timeStamp'] = df.index

    return results_df

results_df = optimizeStorage(df,first_model_hour=0, last_model_hour=8759, setup = sp2)
results_df.to_csv('optimization_resultscase2.csv')


results_df['revenue'] = results_df.Eout * results_df.price - results_df.Ein * results_df.price 
total_revenue = results_df.revenue.sum()

serviceLife = 10
capex = sp2['Storage System Cost']*sp2['Storage Energy']*1000/serviceLife

print('Annual revenue was ${:,.2f}'.format(total_revenue))
print('Annual cost ${:,.2f}'.format(capex))

    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (P) to be 'Any', you can suppress this
    warning by explicitly specifying 'within=Any' to the Param constructor.
    (deprecated in 5.6.9, will be removed in 6.0) (called from
    C:\Users\cd7\AppData\Local\Continuum\anaconda3\envs\py39\lib\site-
    packages\pyomo\core\base\indexed_component.py:707)
    (ordered) position is deprecated.  Please use at()  (deprecated in 6.1,
    will be removed in 7.0) (called from
    C:\Users\cd7\AppData\Local\Temp/ipykernel_10068/1834806405.py:65)
Annual revenue was $7,781,602.62
Annual cost $7,500,000.00


### Case 3: Calculate the projected backcast 2019 revenue of Solar + Storage configuration
Here, two settings are considered: (1) AC coupled (left), (2) DC coupled (right).

AC coupled  | DC coupled
- | - 
![alt](./Picture1.png) | ![alt](./Picture2.png)

Additional Constraints:

For AC coupled:

$\forall t$, $0<=E_t^{SoutG}+E_t^{Sin}<=AC_t$


For DC coupled:

$\forall t$, $0<=E_t^{SoutG}+E_t^{Sin}<=DC_t$

$\forall t$, $0<=E_t^{SoutG}+E_t^{BoutG}<=250$

In [7]:
def optimizeSolStorage(df, first_model_hour=0, last_model_hour=8759, setup=sp3):
    df = df.loc[first_model_hour:last_model_hour, :]

    model = ConcreteModel()

    # Define model parameters
    model.T = Set(doc='hour of year', initialize=df.hour.tolist(), ordered=True)

    model.Rmax = Param(initialize=setup['Storage Capacity'],
                       doc='Max rate of power flow (MW) in or out')
    model.Smax = Param(initialize=setup['Storage Energy'], doc='Max storage (MWh)')
    model.Dmax = Param(initialize=600, doc='Max discharge in 24 hour period')
    model.P = Param(initialize=df['Settlement Point Price'].tolist(), doc='RTMP for each hour')
    eta = 1 # Round trip storage efficiency

    # Charge in from grid, discharge out to grid, and state of charge
    model.GinE = Var(model.T, domain=NonNegativeReals)
    model.EoutG = Var(model.T, domain=NonNegativeReals)
    model.S = Var(model.T, bounds=(0, model.Smax))


    # Solar output to grid
    # Solar output to storage
    model.SinE = Var(model.T, domain=NonNegativeReals)
    model.SoutG = Var(model.T, domain=NonNegativeReals)

    model.ACout = Param(initialize=df.SolarAC.tolist(),doc='Solar AC')

    #Set all constraints
    def storage_state(model, t):
        'Storage changes with flows in/out and efficiency losses'
        # Set first hour state of charge to half of max 
        # this can be change in to 0
        if t == model.T.first():
            return model.S[t] == 0

        else:
            return (model.S[t] == (model.S[t-1] 
                                + (model.GinE[t-1] ) 
                                + (model.SinE[t-1] ) 
                                - (model.EoutG[t-1])))

    model.charge_state = Constraint(model.T, rule=storage_state)

    def discharge_constraint(model, t):
        "Maximum dischage within a single hour"
        return model.EoutG[t] <= model.Rmax

    model.discharge = Constraint(model.T, rule=discharge_constraint)

    def charge_constraint(model, t):
        "Maximum charge within a single hour"
        return (model.GinE[t]+model.SinE[t]) <= model.Rmax

    model.charge = Constraint(model.T, rule=charge_constraint)

    def positive_charge(model, t):
        'Limit discharge to the amount of charge in battery, including losses'
        return model.EoutG[t] <= model.S[t] 
    model.positive_charge = Constraint(model.T, rule=positive_charge)


    def inverter_limit(model, t):
        'Limit inverter output'
        return model.SoutG[t] + model.SinE[t]<=model.ACout.extract_values()[None][t]
 
    model.inverter_limit = Constraint(model.T, rule=inverter_limit)



    # Define the total income, expenses, and profit
    income = sum(df.loc[t, 'Settlement Point Price'] * (model.EoutG[t]+ model.SoutG[t])*np.sqrt(eta) for t in model.T)
    expenses = sum(df.loc[t, 'Settlement Point Price'] * model.GinE[t]/np.sqrt(eta) for t in model.T)
    profit = income - expenses
    model.objective = Objective(expr=profit, sense=maximize)

    # Solve the model
    solver = SolverFactory('glpk')
    solver.solve(model)

    hours = range(model.T[first_model_hour + 1], model.T[last_model_hour + 1] + 1)
    GinE = [value(model.GinE[i]) for i in hours]
    EoutG = [value(model.EoutG[i]) for i in hours]
    SinE = [value(model.SinE[i]) for i in hours]
    SoutG = [value(model.SoutG[i]) for i in hours]    
    Price = [model.P.extract_values()[None][i] for i in hours]
    charge_state = [value(model.S[i]) for i in hours]

    df_dict = dict(
        hour = hours,
        GinE = GinE,
        EoutG = EoutG,
        SinE = SinE,
        SoutG = SoutG,
        price = Price,
        charge_state = charge_state
    )

    results_df = pd.DataFrame(df_dict)

    results_df['timeStamp'] = df.index

    return results_df

results_df = optimizeSolStorage(df,first_model_hour=0, last_model_hour=8759, setup = sp3)
results_df.to_csv('optimization_resultscase3AC.csv')


results_df['revenue'] = (results_df.EoutG+ results_df.SoutG)* results_df.price - (results_df.GinE) * results_df.price
total_revenue = results_df.revenue.sum()

serviceLifeStorange = 10
serviceLifeSolar = 30

capex = sp3['Storage System Cost']*sp3['Storage Energy']*1000/serviceLifeStorange + sp3['Solar Size']*sp3['Solar System Cost']*1000000/serviceLifeSolar


print('Annual revenue was ${:,.2f}'.format(total_revenue))
print('Annual cost ${:,.2f}'.format(capex))

    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (ACout) to be 'Any', you can suppress this
    warning by explicitly specifying 'within=Any' to the Param constructor.
    (deprecated in 5.6.9, will be removed in 6.0) (called from
    C:\Users\cd7\AppData\Local\Continuum\anaconda3\envs\py39\lib\site-
    packages\pyomo\core\base\indexed_component.py:707)
    (ordered) position is deprecated.  Please use at()  (deprecated in 6.1,
    will be removed in 7.0) (called from
    C:\Users\cd7\AppData\Local\Temp/ipykernel_10068/3261308663.py:81)
Annual revenue was $53,361,535.00
Annual cost $19,500,000.00


In [8]:
def optimizeSolStorage(df, first_model_hour=0, last_model_hour=8759, setup=sp3):
    df = df.loc[first_model_hour:last_model_hour, :]

    model = ConcreteModel()

    # Define model parameters
    model.T = Set(doc='hour of year', initialize=df.hour.tolist(), ordered=True)

    model.Rmax = Param(initialize=setup['Storage Capacity'],
                       doc='Max rate of power flow (MW) in or out')
    model.Smax = Param(initialize=setup['Storage Energy'], doc='Max storage (MWh)')
    model.Dmax = Param(initialize=600, doc='Max discharge in 24 hour period')
    model.P = Param(initialize=df['Settlement Point Price'].tolist(), doc='RTMP for each hour')
    eta = 1 # Round trip storage efficiency

    # Charge in from grid, discharge out to grid, and state of charge
    model.GinE = Var(model.T, domain=NonNegativeReals)
    model.EoutG = Var(model.T, domain=NonNegativeReals)
    model.S = Var(model.T, bounds=(0, model.Smax))


    # Solar output to grid
    # Solar output to storage
    model.SinE = Var(model.T, domain=NonNegativeReals)
    model.SoutG = Var(model.T, domain=NonNegativeReals)

    model.DCout = Param(initialize=df.SolarDC.tolist(),doc='Solar DC')

    #Set all constraints
    def storage_state(model, t):
        'Storage changes with flows in/out and efficiency losses'
        # Set first hour state of charge to half of max 
        # this can be change in to 0
        if t == model.T.first():
            # return model.S[t] == model.Smax / 2
            return model.S[t] == 0

        else:
            return (model.S[t] == (model.S[t-1] 
                                + (model.GinE[t-1] ) 
                                + (model.SinE[t-1] ) 
                                - (model.EoutG[t-1])))

    model.charge_state = Constraint(model.T, rule=storage_state)

    def discharge_constraint(model, t):
        "Maximum dischage within a single hour"
        return model.EoutG[t] <= model.Rmax

    model.discharge = Constraint(model.T, rule=discharge_constraint)

    def charge_constraint(model, t):
        "Maximum charge within a single hour"
        return  (model.GinE[t]+model.SinE[t]) <= model.Rmax

    model.charge = Constraint(model.T, rule=charge_constraint)

    
    def positive_charge(model, t):
        'Limit discharge to the amount of charge in battery, including losses'
        return model.EoutG[t] <= model.S[t] 
    model.positive_charge = Constraint(model.T, rule=positive_charge)

    def DC_limit(model, t):
        'Limit DC output'
        return (model.SoutG[t] + model.SinE[t])<=model.DCout.extract_values()[None][t]

    model.DC_limit = Constraint(model.T, rule=DC_limit)

    def inverter_limit(model, t):
        'Limit inverter output'
        return model.SoutG[t] + model.EoutG[t]<=setup['Inverter Capacity']

    model.inverter_limit = Constraint(model.T, rule=inverter_limit)

    # Define the total income, expenses, and profit
    income = sum(df.loc[t, 'Settlement Point Price'] * (model.EoutG[t]+ model.SoutG[t]) for t in model.T)
    expenses = sum(df.loc[t, 'Settlement Point Price'] * model.GinE[t] for t in model.T)
    profit = income - expenses
    model.objective = Objective(expr=profit, sense=maximize)

    # Solve the model
    solver = SolverFactory('glpk')
    solver.solve(model)

    hours = range(model.T[first_model_hour + 1], model.T[last_model_hour + 1] + 1)
    GinE = [value(model.GinE[i]) for i in hours]
    EoutG = [value(model.EoutG[i]) for i in hours]
    SinE = [value(model.SinE[i]) for i in hours]
    SoutG = [value(model.SoutG[i]) for i in hours]    
    Price = [model.P.extract_values()[None][i] for i in hours]
    charge_state = [value(model.S[i]) for i in hours]

    df_dict = dict(
        hour = hours,
        GinE = GinE,
        EoutG = EoutG,
        SinE = SinE,
        SoutG = SoutG,
        price = Price,
        charge_state = charge_state
    )

    results_df = pd.DataFrame(df_dict)

    results_df['timeStamp'] = dti

    return results_df

results_df = optimizeSolStorage(df,first_model_hour=0, last_model_hour=8759, setup = sp3)

results_df.to_csv('optimization_resultscase3DC.csv')


results_df['revenue'] = (results_df.EoutG+ results_df.SoutG)* results_df.price - (results_df.GinE) * results_df.price  
total_revenue = results_df.revenue.sum()

serviceLifeStorange = 10
serviceLifeSolar = 30

capex = sp3['Storage System Cost']*sp3['Storage Energy']*1000/serviceLifeStorange + sp3['Solar Size']*sp3['Solar System Cost']*1000000/serviceLifeSolar


print('Annual revenue was ${:,.2f}'.format(total_revenue))
print('Annual cost ${:,.2f}'.format(capex))



    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (DCout) to be 'Any', you can suppress this
    warning by explicitly specifying 'within=Any' to the Param constructor.
    (deprecated in 5.6.9, will be removed in 6.0) (called from
    C:\Users\cd7\AppData\Local\Continuum\anaconda3\envs\py39\lib\site-
    packages\pyomo\core\base\indexed_component.py:707)
    (ordered) position is deprecated.  Please use at()  (deprecated in 6.1,
    will be removed in 7.0) (called from
    C:\Users\cd7\AppData\Local\Temp/ipykernel_10068/672090882.py:86)
Annual revenue was $49,247,600.45
Annual cost $19,500,000.00


##  Using Gurobipy

(1) for the GCV standalone storage system, it is an optimization problem (energy arbitrage).

In [9]:
import gurobipy as gp
from gurobipy import GRB

In [10]:
# Parameters

nhours = 8760
StorageCapacity = 50
StorageEnergy = 200
MaxDischarge = 600
rtm = df['Settlement Point Price'].tolist()
timeh = range(nhours)

In [11]:
eta =1
# Create a new model
m = gp.Model ("solarPlan")
# Create variables
eOut = m.addVars(timeh,  vtype = GRB.CONTINUOUS, name ="eout")
eIn = m.addVars (timeh,  vtype = GRB.CONTINUOUS , name ="ein")
soc = m.addVars (timeh, ub=StorageEnergy, vtype = GRB.CONTINUOUS, name ="soc")

Set parameter Username
Academic license - for non-commercial use only - expires 2022-11-17


## Set constraints

In [12]:
start_state = m.addConstrs((soc[t] == 0 for t in timeh  if t==0),'SOCst')
storage_state = m.addConstrs(( soc[t] == (soc[t-1] 
                                + (eIn[t-1] * np.sqrt(eta)) 
                                - (eOut[t-1] / np.sqrt(eta))) for t in timeh[1:]),'SOC')

In [13]:
discharge_state = m.addConstrs((eOut[t] <= StorageCapacity for t in timeh),'discharge_st')

charge_state = m.addConstrs( (eIn[t] <= StorageCapacity for t in timeh),'charge_st')

positive_charge = m.addConstrs( (eOut[t] <= soc[t]/np.sqrt(eta) for t in timeh),'poscharge')

In [14]:
### Define the battery income, expenses, and profit
# Set objective
m.setObjective(sum(rtm[t]*(eOut[t] - eIn[t]) for t in timeh), GRB.MAXIMIZE)

In [15]:
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 35040 rows, 26280 columns and 70077 nonzeros
Model fingerprint: 0xd456e1a4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 9e+03]
  Bounds range     [2e+02, 2e+02]
  RHS range        [5e+01, 5e+01]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 17527 rows and 9 columns
Presolve time: 0.14s
Presolved: 17513 rows, 26271 columns, 52542 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.627e+04
 Factor NZ  : 2.188e+05 (roughly 20 MB of memory)
 Factor Ops : 3.439e+06 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -7.17506212e+02  1.64111291e+10  3.03e+01 9.02e+02  5.25e+05     0s
   1   9.23833172e+05  

### Case 3 AC

In [16]:
m = gp.Model ("solarPlanAC")

In [17]:
# add additional Parameters
solarCost = 1.25
batteryCost = 350
ACout =  df.SolarAC.tolist()

In [18]:
# Create variables
GInB = m.addVars(timeh, lb=0, vtype = GRB.CONTINUOUS , name ="GinB")
SinB = m.addVars(timeh, lb=0, vtype = GRB.CONTINUOUS, name ="SinB")
BoutG = m.addVars(timeh, lb=0, vtype = GRB.CONTINUOUS, name ="BoutG")
SoutG = m.addVars(timeh, lb=0, vtype = GRB.CONTINUOUS, name ="SoutG")
soc = m.addVars (timeh, lb=0, ub=StorageEnergy, vtype = GRB.CONTINUOUS, name ="soc")

### constraints

In [19]:
start_state = m.addConstrs((soc[t] == 0 for t in timeh  if t==0),'SOCst')
storage_state = m.addConstrs(( soc[t] == (soc[t-1] 
                                + (GInB[t-1]+SinB[t-1])*np.sqrt(eta) 
                                - (BoutG[t-1] / np.sqrt(eta))) for t in timeh[1:]),'SOC')

In [20]:
discharge_state = m.addConstrs((BoutG[t]/np.sqrt(eta) <= StorageCapacity for t in timeh),'discharge_st')

charge_state = m.addConstrs( ((GInB[t]+SinB[t])*np.sqrt(eta) <= StorageCapacity for t in timeh),'charge_st')

positive_charge = m.addConstrs((BoutG[t]/np.sqrt(eta) <= soc[t] for t in timeh),'poscharge')

inverter_lim = m.addConstrs((SoutG[t]+SinB[t] <= ACout[t]  for t in timeh),'poscharge')

In [21]:
### Define the battery income, expenses, and profit
# Set objective
m.setObjective(sum(rtm[t]*(BoutG[t]+ SoutG[t]-GInB[t]) for t in timeh), GRB.MAXIMIZE)
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 43800 rows, 43800 columns and 105116 nonzeros
Model fingerprint: 0xe0494349
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 9e+03]
  Bounds range     [2e+02, 2e+02]
  RHS range        [3e-07, 3e+02]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 26287 rows and 17529 columns
Presolve time: 0.18s
Presolved: 17513 rows, 26271 columns, 52542 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.627e+04
 Factor NZ  : 2.188e+05 (roughly 20 MB of memory)
 Factor Ops : 3.439e+06 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.55802907e+07  1.64560843e+10  7.00e+01 9.02e+02  5.25e+05     0s
   1   4.65037726e

### Case 3 DC

In [22]:
m = gp.Model ("solarPlanDC")
# add additional Parameters
DCout =  df.SolarDC.tolist()
InverterCapacity = 250

In [23]:
# Create variables
GInB = m.addVars(timeh, lb=0, vtype = GRB.CONTINUOUS , name ="GinB")
SinB = m.addVars(timeh, lb=0, vtype = GRB.CONTINUOUS, name ="SinB")
BoutG = m.addVars(timeh, lb=0, vtype = GRB.CONTINUOUS, name ="BoutG")
SoutG = m.addVars(timeh, lb=0, vtype = GRB.CONTINUOUS, name ="SoutG")
soc = m.addVars (timeh, lb=0, ub=StorageEnergy, vtype = GRB.CONTINUOUS, name ="soc")

In [24]:
start_state = m.addConstrs((soc[t] == 0 for t in timeh  if t==0),'SOCst')
storage_state = m.addConstrs(( soc[t] == (soc[t-1] 
                                + (GInB[t-1]+SinB[t-1])*np.sqrt(eta) 
                                - (BoutG[t-1] / np.sqrt(eta))) for t in timeh[1:]),'SOC')

In [25]:
discharge_state = m.addConstrs((BoutG[t]/np.sqrt(eta) <= StorageCapacity for t in timeh),'discharge_st')

charge_state = m.addConstrs( ((GInB[t]+SinB[t])*np.sqrt(eta) <= StorageCapacity for t in timeh),'charge_st')

positive_charge = m.addConstrs((BoutG[t]/np.sqrt(eta) <= soc[t] for t in timeh),'poscharge')

inverter_lim = m.addConstrs((SoutG[t]+SinB[t] <= DCout[t]  for t in timeh),'poscharge')

dc_lim =  m.addConstrs((SoutG[t]+BoutG[t] <= InverterCapacity  for t in timeh),'poscharge')

In [26]:
### Define the battery income, expenses, and profit
# Set objective
m.setObjective(sum(rtm[t]*(BoutG[t]+ SoutG[t]-GInB[t]) for t in timeh), GRB.MAXIMIZE)
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 52560 rows, 43800 columns and 122636 nonzeros
Model fingerprint: 0xd7b64d40
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 9e+03]
  Bounds range     [2e+02, 2e+02]
  RHS range        [3e-07, 3e+02]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 29938 rows and 14123 columns
Presolve time: 0.19s
Presolved: 22622 rows, 29677 columns, 64463 nonzeros

Ordering time: 0.01s

Barrier statistics:
 AA' NZ     : 3.649e+04
 Factor NZ  : 2.377e+05 (roughly 23 MB of memory)
 Factor Ops : 3.413e+06 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   5.86444851e+07  1.71058315e+10  2.85e+02 2.23e+03  1.22e+06     0s
   1   3.19833119e

## OPTIMIZING ELECTRICITY TRANSMISSION COST

In [28]:
from gurobipy import *
#sets
plants=['Plant A','Plant B','Plant C' ]
cities =['NewYork','Boston','Philadelphia','NewJersy']
I = range(len(plants))
J = range(len(cities))

#data
Costs= [[8,12,7,3],
    [4,18,6,9],
    [14,2,10,15]]

Demand = [50,30,70,20]
Supply = [25,60,50]


m = Model('Trans Model')

X={}
for i in I:
    for j in J:
        X[i,j]=m.addVar()
        

m.setObjective(quicksum(Costs[i][j]*X[i,j] for i in I for j in J), GRB.MINIMIZE)

for j in J:
    m.addConstr(quicksum(X[i,j] for i in I) >= Demand[j])

for i in I:
    m.addConstr(quicksum(X[i,j] for j in J) <= Supply[i])

m.optimize()

for j in J:
    print('-'*30)
    print("cities:",j,"Demand:",Demand[j])
    print("-"*30)
    for i in I:
        print("     plants", i, X[i,j].x)



Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 7 rows, 12 columns and 24 nonzeros
Model fingerprint: 0xb4560778
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 7e+01]
Presolve time: 0.01s
Presolved: 7 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.700000e+02   0.000000e+00      0s

Solved in 6 iterations and 0.02 seconds (0.00 work units)
Infeasible model
------------------------------
cities: 0 Demand: 50
------------------------------
     plants 0 0.0
     plants 1 50.0
     plants 2 0.0
------------------------------
cities: 1 Demand: 30
------------------------------
     plants 0 0.0
     plants 1 0.0
     plants 2 30.0
------------------------------
cities: 2 Demand: 70
-------------------------